In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
df=pd.read_csv(r"..\csv_files\titanic\train.csv")
test=pd.read_csv(r"..\csv_files\titanic\test.csv")

In [18]:
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Cabin'] = df['Cabin'].notnull().astype(int)  # Convert to binary feature
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

In [19]:
test['Age'].fillna(test['Age'].median(), inplace=True)
test['Cabin'] = test['Cabin'].notnull().astype(int)  # Convert to binary feature
test['Embarked'].fillna(test['Embarked'].mode()[0], inplace=True)

In [20]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1


In [21]:
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1


In [22]:
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
df['Title'] = df['Title'].replace(['Mme'], 'Mrs')
df['Title'] = df['Title'].replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer')
df['Title'] = df['Title'].replace(['Don', 'Dona', 'Lady', 'Sir', 'Countess', 'Jonkheer'], 'Royalty')


<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\Users\hp\AppData\Local\Temp\ipykernel_13496\1450961300.py:1: SyntaxWarning: invalid escape sequence '\.'
  df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


In [23]:
test['Title'] = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test['Title'] = test['Title'].replace(['Mlle', 'Ms'], 'Miss')
test['Title'] = test['Title'].replace(['Mme'], 'Mrs')
test['Title'] = test['Title'].replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer')
test['Title'] = test['Title'].replace(['Don', 'Dona', 'Lady', 'Sir', 'Countess', 'Jonkheer'], 'Royalty')


<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\Users\hp\AppData\Local\Temp\ipykernel_13496\2154131037.py:1: SyntaxWarning: invalid escape sequence '\.'
  test['Title'] = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


In [24]:
df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)


In [25]:
Id = pd.DataFrame({'PassengerId': test['PassengerId']})
test.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)


In [26]:
df = pd.get_dummies(df, columns=['Sex', 'Embarked', 'Title'], drop_first=True)


In [27]:
test = pd.get_dummies(test, columns=['Sex', 'Embarked', 'Title'], drop_first=True)


In [28]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['Age', 'Fare', 'FamilySize']] = scaler.fit_transform(df[['Age', 'Fare', 'FamilySize']])
test[['Age', 'Fare', 'FamilySize']] = scaler.fit_transform(test[['Age', 'Fare', 'FamilySize']])


In [29]:
X = df.drop("Survived", axis=1)
y = df["Survived"]

In [30]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
param_dist = {
    'n_estimators': randint(50, 200),  # Randomly select between 50 and 200 trees
    'max_depth': [None, 10, 20, 30],    # Max depth options
    'min_samples_split': randint(2, 10),  # Randomly select between 2 and 10
    'min_samples_leaf': randint(1, 10),    # Randomly select between 1 and 10
    'max_features': ['auto', 'sqrt', 'log2'],  # Feature selection methods
    'bootstrap': [True, False]               # Bootstrap sampling
}
rf = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, 
                                   n_iter=100, cv=5, n_jobs=-1, random_state=42, verbose=2)
random_search.fit(X, y)
print(f'Best Hyperparameters: {random_search.best_params_}')
best_model = random_search.best_estimator_
y_pred = best_model.predict(test)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Hyperparameters: {'bootstrap': False, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 7, 'min_samples_split': 5, 'n_estimators': 89}


c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
160 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
95 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 66

In [31]:
submission = pd.DataFrame({
    'PassengerId': Id['PassengerId'],
    'Survived': y_pred
    })
submission.to_csv('submission.csv', index=False)